<a href="https://colab.research.google.com/github/flyinacres/fastai-coursework/blob/main/collab_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# installing/upgrading fastai2
!pip install --upgrade fastai

from fastai.vision.all import *

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.3 MB/s eta 0:00:00


In [2]:
%matplotlib inline
from fastai.tabular.all import *
from fastai.collab import *

In [3]:
path = untar_data(URLs.ML_100k)

In [4]:
ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None,
                      usecols=(0,1,2), names=['user','movie','rating'])
ratings.head()

,user,movie,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [5]:
movies = pd.read_csv(path/'u.item',  delimiter='|', encoding='latin-1',
                     usecols=(0,1), names=('movie','title'), header=None)
movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [6]:
ratings = ratings.merge(movies)
ratings.head()

,user,movie,rating,title
0,196,242,3,Kolya (1996)
1,63,242,3,Kolya (1996)
2,226,242,5,Kolya (1996)
3,154,242,3,Kolya (1996)
4,306,242,5,Kolya (1996)


In [7]:
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=64)

In [8]:
dls.show_batch()

,user,title,rating
0,62,Full Metal Jacket (1987),3
1,751,"Sex, Lies, and Videotape (1989)",4
2,887,Snow White and the Seven Dwarfs (1937),5
3,92,To Die For (1995),4
4,749,Searching for Bobby Fischer (1993),4
5,94,"Maltese Falcon, The (1941)",5
6,327,Toy Story (1995),4
7,343,Priest (1994),5
8,326,Aliens (1986),2
9,504,Amityville II: The Possession (1982),3


In [9]:
learn = collab_learner(dls, n_factors=50, y_range=(0, 5.5))

In [10]:
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.954045,0.935689,00:10
1,0.875464,0.860040,00:10
2,0.728251,0.824837,00:10
3,0.593251,0.811040,00:11
4,0.489497,0.812322,00:10


Nice to actually get a learner running on Colab again, after all the challenges with the text examples...

In [11]:
g = ratings.groupby('title')['rating'].count()
top_movies = g.sort_values(ascending=False).index.values[:1000]
top_movies[:10]

array(['Star Wars (1977)', 'Contact (1997)', 'Fargo (1996)',
       'Return of the Jedi (1983)', 'Liar Liar (1997)',
       'English Patient, The (1996)', 'Scream (1996)', 'Toy Story (1995)',
       'Air Force One (1997)', 'Independence Day (ID4) (1996)'],
      dtype=object)

In [12]:
movie_bias = learn.model.bias(top_movies, is_item=True)
movie_bias.shape

torch.Size([1000])

mean_ratings is a Pandas Series--a one dimensional ndarray with axis labels.

ratings is a Pandas DataFrame, a table.  In this case with numeric indicees

movie ratings is a list

In [13]:
mean_ratings = ratings.groupby('title')['rating'].mean()
movie_ratings = [(b, i, mean_ratings.loc[i]) for i,b in zip(top_movies,movie_bias)]

https://www.w3schools.com/python/python_lambda.asp
https://www.geeksforgeeks.org/python-sorted-function/

movie ratings has multiple dimensions, so this ensures it is sorted by the zeroth dimension, which should be the bias

In [14]:
item0 = lambda o:o[0]
sorted(movie_ratings, key=item0)[:15]

[(tensor(-0.3608),
  'Children of the Corn: The Gathering (1996)',
  1.3157894736842106),
 (tensor(-0.3067), 'Mortal Kombat: Annihilation (1997)', 1.9534883720930232),
 (tensor(-0.2716),
  'Lawnmower Man 2: Beyond Cyberspace (1996)',
  1.7142857142857142),
 (tensor(-0.2626), 'Striptease (1996)', 2.2388059701492535),
 (tensor(-0.2428), 'Beautician and the Beast, The (1997)', 2.313953488372093),
 (tensor(-0.2422), 'Free Willy 3: The Rescue (1997)', 1.7407407407407407),
 (tensor(-0.2369), 'Island of Dr. Moreau, The (1996)', 2.1578947368421053),
 (tensor(-0.2361), 'Barb Wire (1996)', 1.9333333333333333),
 (tensor(-0.2283), 'Flintstones, The (1994)', 2.064516129032258),
 (tensor(-0.2231), 'Cable Guy, The (1996)', 2.339622641509434),
 (tensor(-0.2214), 'Crow: City of Angels, The (1996)', 1.9487179487179487),
 (tensor(-0.2156), 'Escape from L.A. (1996)', 2.4615384615384617),
 (tensor(-0.2146), 'Home Alone 3 (1997)', 1.894736842105263),
 (tensor(-0.2118), 'Speed 2: Cruise Control (1997)', 2.13

In [15]:
sorted(movie_ratings, key=lambda o: o[0], reverse=True)[:15]

[(tensor(0.6113), 'Shawshank Redemption, The (1994)', 4.445229681978798),
 (tensor(0.5964), 'Titanic (1997)', 4.2457142857142856),
 (tensor(0.5488), 'L.A. Confidential (1997)', 4.161616161616162),
 (tensor(0.5325), 'Apt Pupil (1998)', 4.1),
 (tensor(0.5296), 'Rear Window (1954)', 4.3875598086124405),
 (tensor(0.5258), "Schindler's List (1993)", 4.466442953020135),
 (tensor(0.5053), 'Boot, Das (1981)', 4.203980099502488),
 (tensor(0.5041), 'Star Wars (1977)', 4.3584905660377355),
 (tensor(0.5013), 'Silence of the Lambs, The (1991)', 4.28974358974359),
 (tensor(0.4722), 'Good Will Hunting (1997)', 4.262626262626263),
 (tensor(0.4626), 'Close Shave, A (1995)', 4.491071428571429),
 (tensor(0.4620), 'Casablanca (1942)', 4.45679012345679),
 (tensor(0.4609), 'Godfather, The (1972)', 4.283292978208232),
 (tensor(0.4573), 'Wrong Trousers, The (1993)', 4.466101694915254),
 (tensor(0.4499), 'Vertigo (1958)', 4.251396648044692)]

In [16]:
??ratings

In [18]:
mean_ratings.shape

(1664,)

In [19]:
mean_ratings[:10]

title
'Til There Was You (1997)                      2.333333
1-900 (1994)                                   2.600000
101 Dalmatians (1996)                          2.908257
12 Angry Men (1957)                            4.344000
187 (1997)                                     3.024390
2 Days in the Valley (1996)                    3.225806
20,000 Leagues Under the Sea (1954)            3.500000
2001: A Space Odyssey (1968)                   3.969112
3 Ninjas: High Noon At Mega Mountain (1998)    1.000000
39 Steps, The (1935)                           4.050847
Name: rating, dtype: float64

In [ ]:
??mean_ratings